<a href="https://colab.research.google.com/github/Aurelio-Naufal/KI_Task_Aurelio-Naufal-Effendy/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Aurelio Naufal Effendy

NPM : 2106638526

Kelas : Komputasi Intelegensia

In [3]:
!pip install indonlp -q
!pip install transformers -q
!pip install torch -q
!pip install datasets -q

# Sentimen Analysis Model Using bert-base-indonesian-1.5G-sentiment-analysis-smsa model

In [4]:
from transformers import pipeline

# Load the fine-tuned BERT model for Indonesian sentiment analysis
sentiment_analysis = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

# Test the model on an example sentence
result = sentiment_analysis("Film ini sangat bagus, saya menyukainya!")
print(f"Sentiment: {result}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Sentiment: [{'label': 'Positive', 'score': 0.999775230884552}]


In [5]:
# Test on another sentences
result = sentiment_analysis("Film ini sangat membosankan")
print(f"Sentiment: {result}")

Sentiment: [{'label': 'Negative', 'score': 0.9998849630355835}]


# Check model's accuracy using a synthetic dataset

In [6]:
from datasets import load_dataset
from tqdm import tqdm

# Load the Indonesian Sentiment dataset
dataset = load_dataset("sepidmnorozy/Indonesian_sentiment")

# Split the dataset into training and test sets
train_data = dataset['train']
test_data = dataset['test']

num_data_points = len(test_data)
print(f"Number of data points in the train dataset: {num_data_points}")

# Print an example
print(train_data[0])

Number of data points in the train dataset: 2266
{'label': 1, 'text': 'bubur ayam yang lumayan rekomendasi di sekitaran bandung , tempat nya strategis mudah dicari , harga nya tidak merogoh kantong , tempat nya selalu ramai didatangi pengunjung setiap hari kerja maupun akhir pekan karena rasanya yang enak .'}


In [8]:
test_dataset = test_data[:500] #use 500 data only

In [9]:
def predict_sentiments(texts):
    # Tokenize and make predictions for each text
    predictions = []
    for text in tqdm(texts):
        sentiment = sentiment_analysis(text)
        predictions.append(sentiment)
    return predictions

# Check Model's accuracy

In [10]:
label_mapping = {
    0: 'Negative',
    1: 'Positive',
}

In [13]:
true_labels = [label_mapping[label] for label in test_dataset['label']]

In [14]:
# Evaluate the model on the test split of the Indonesian Sentiment dataset
predicted_labels = predict_sentiments(test_dataset['text'])

100%|██████████| 500/500 [01:50<00:00,  4.51it/s]


In [15]:
extracted_labels = [pred[0]['label'] for pred in predicted_labels]

In [16]:
import numpy as np
accuracy = sum(np.array(extracted_labels) == np.array(true_labels)) / len(true_labels)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9920


The accuracy is pretty good, reaching 99.2%

# Using Attention Transformer

In [17]:
import torch
import numpy as np
from tqdm import tqdm
from transformers import BertModel, BertTokenizer
from transformers import Trainer, TrainingArguments
import torch.nn as nn

# Load the pre-trained BERT model and tokenizer
model_name = "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa"
tokenizer = BertTokenizer.from_pretrained(model_name)

class CustomBERTWithAttention(nn.Module):
    def __init__(self):
        super(CustomBERTWithAttention, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.attention = nn.MultiheadAttention(embed_dim=768, num_heads=8)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, 2)  # 2 classes: positive and negative

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)

        # Use the mean of hidden states as input to the attention layer
        attn_output, _ = self.attention(hidden_states, hidden_states, hidden_states)
        pooled_output = torch.mean(attn_output, dim=1)  # Aggregate the attention output

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Initialize the custom model
custom_model = CustomBERTWithAttention()

# Move the model to the appropriate device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_model.to(device)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CustomBERTWithAttention(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [18]:
# Function to predict sentiments using the custom model
def predict_sentiments(texts):
    # Tokenize inputs
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

    # Get predictions
    with torch.no_grad():
        outputs = custom_model(inputs['input_ids'], inputs['attention_mask'])

    # Convert logits to probabilities
    probabilities = torch.softmax(outputs, dim=1)

    # Get predicted class labels
    predicted_labels = torch.argmax(probabilities, dim=1).cpu().numpy()
    return predicted_labels

# Check Model's accuracy with attention mask

In [20]:
# Get predicted labels using the defined function
predicted_labels_with_attention = predict_sentiments(test_dataset['text'])
predicted_labels_with_attention

In [22]:
extracted_labels_with_attention = [label_mapping[label] for label in predicted_labels_with_attention]

In [24]:
# Calculate accuracy
accuracy_with_attention = sum(np.array(extracted_labels_with_attention) == np.array(true_labels)) / len(true_labels)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9920


It got the exact same accuracy